# Instalando as bibliotecas usadas

Algumas das bibliotecas usadas não fazem parte da distribuição padrão do Python.

A célula abaixo realiza a instalação delas para garantir o funcionamento do código.

In [ ]:
!pip install pandas
!pip install googledrivedownloader

# Questão 1) Coleta de dados

## Baixa microdados do ENADE do Google Drive

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

access_link_microdados_zip = 'https://drive.google.com/file/d/1buaKZzayV2tsYXo_Q6xM0jk57O03h5sI/view?usp=sharing'
drive_ID_covid_dados = '1buaKZzayV2tsYXo_Q6xM0jk57O03h5sI'

# Se o arquivo já existe, deleto e faço o download novamente
if (os.path.isfile('./Microdados ENADE.zip')):
    os.remove('./Microdados ENADE.zip')

gdd.download_file_from_google_drive(drive_ID_covid_dados,'./Microdados ENADE.zip', unzip = True)

Crio uma cópia dos dados .csv para a pasta Downloads para que o KNIME possa lê-los

In [ ]:
import shutil

# Copio pra pasta de downloads para que o KNIME possa ler
def copyToDownloadFolder(file_path):
    shutil.copy(file_path, os.path.join(os.getenv('USERPROFILE'), 'Downloads'))

endereco_dados_2017 = os.path.join(os.getcwd(), 'Microdados ENADE', '2017', '3.DADOS', 'MICRODADOS_ENADE_2017.txt')
endereco_dados_2018 = os.path.join(os.getcwd(), 'Microdados ENADE', '2018', '3.DADOS', 'MICRODADOS_ENADE_2018.txt')
endereco_dados_2019 = os.path.join(os.getcwd(), 'Microdados ENADE', '2019', '3.DADOS', 'MICRODADOS_ENADE_2019.txt')

copyToDownloadFolder(endereco_dados_2017)
copyToDownloadFolder(endereco_dados_2018)
copyToDownloadFolder(endereco_dados_2019)

# Questão 3) Cria o banco de dados

Reaproveitando o código do trabalho em grupo para criar conexão com o Banco de Dados

In [ ]:
import sqlite3
from sqlite3 import Error
import os

#https://www.sqlitetutorial.net/sqlite-python/

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

def getPath():
    return os.path.join(os.getcwd(), 'DW prova individual BD.db')

if __name__ == '__main__':
    conn = create_connection(getPath())

## Crio as tabelas seguindo a Modelagem Dimensional

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    database = getPath()

    # Crio a tabela fato ENADE
    sql_create_table_ENADE = """
        CREATE TABLE IF NOT EXISTS ENADE(
            FK_tempo_ID integer,
            FK_info_curso_ID integer,
            FK_estudante_ID integer,
            FK_num_item_ID integer,
            FK_vetor_ID integer,
            FK_tipo_presenca_ID integer,
            FK_tipo_sit_quest_disc_ID integer,
            FK_nota_fg_ce_ID integer,
            FK_questio_percep_prova_ID integer,
            FK_questio_estudante_ID integer,
            FOREIGN KEY (FK_tempo_ID) REFERENCES Tempo(tempo_ID),
            FOREIGN KEY (FK_info_curso_ID) REFERENCES Info_Curso(info_curso_ID),
            FOREIGN KEY (FK_estudante_ID) REFERENCES Estudante(estudante_ID),
            FOREIGN KEY (FK_num_item_ID) REFERENCES Num_Item(num_item_ID),
            FOREIGN KEY (FK_vetor_ID) REFERENCES Vetores(vetor_ID),
            FOREIGN KEY (FK_tipo_presenca_ID) REFERENCES Tipo_Presenca(tipo_presenca_ID),
            FOREIGN KEY (FK_tipo_sit_quest_disc_ID) REFERENCES Tipo_Sit_Quest_Disc(tipo_sit_quest_disc_ID),
            FOREIGN KEY (FK_nota_fg_ce_ID) REFERENCES Nota_FG_CE(nota_fg_ce_ID),
            FOREIGN KEY (FK_questio_percep_prova_ID) REFERENCES QuestioPercepProva(questio_percep_prova_ID),
            FOREIGN KEY (FK_questio_estudante_ID) REFERENCES Questio_Estudante(questio_estudante_ID),
            PRIMARY KEY (FK_tempo_ID, FK_info_curso_ID, FK_estudante_ID, FK_num_item_ID, FK_vetor_ID, FK_tipo_presenca_ID, FK_tipo_sit_quest_disc_ID, FK_nota_fg_ce_ID, FK_questio_percep_prova_ID, FK_questio_estudante_ID)
        );"""

    # Crio a tabela dimensão Tempo
    sql_create_table_Tempo = """
        CREATE TABLE IF NOT EXISTS Tempo(
            tempo_ID integer PRIMARY KEY AUTOINCREMENT,
            NU_ANO integer,
            ANO_FIM_EM integer,
            ANO_IN_GRAD integer
        );"""

    # Crio a tabela dimensão Info_Curso
    sql_create_table_Info_Curso = """
        CREATE TABLE IF NOT EXISTS Info_Curso(
            info_curso_ID integer PRIMARY KEY AUTOINCREMENT,
            CO_IES integer,
            CO_CATEGAD integer,
            CO_ORGACAD integer,
            CO_GRUPO integer,
            CO_CURSO integer,
            CO_MODALIDADE integer,
            CO_MUNIC_CURSO integer,
            CO_UF_CURSO integer,
            CO_REGIAO_CURSO integer
        );"""

    # Crio a tabela dimensão Estudante
    sql_create_table_Estudante = """
        CREATE TABLE IF NOT EXISTS Estudante(
            estudante_ID integer PRIMARY KEY AUTOINCREMENT,
            NU_IDADE integer,
            TP_SEXO text,
            CO_TURNO_GRADUACAO integer,
            TP_INSCRICAO_ADM integer,
            TP_INSCRICAO integer
        );"""

    # Crio a tabela dimensão NU_Item
    sql_create_table_NU_Item = """
        CREATE TABLE IF NOT EXISTS Num_Item(
            num_item_ID integer PRIMARY KEY AUTOINCREMENT,
            NU_ITEM_OFG integer,
            NU_ITEM_OFG_Z integer,
            NU_ITEM_OFG_X integer,
            NU_ITEM_OFG_N integer,
            NU_ITEM_OCE integer,
            NU_ITEM_OCE_Z integer,
            NU_ITEM_OCE_X integer,
            NU_ITEM_OCE_N integer
        );"""

    # Crio a tabela dimensão Vetores
    sql_create_table_Vetores = """
        CREATE TABLE IF NOT EXISTS Vetores(
            vetor_ID integer PRIMARY KEY AUTOINCREMENT,
            DS_VT_GAB_OFG_ORIG text,
            DS_VT_GAB_OFG_FIN text,
            DS_VT_GAB_OCE_ORIG text,
            DS_VT_GAB_OCE_FIN text,
            DS_VT_ESC_OFG text,
            DS_VT_ACE_OFG integer,
            DS_VT_ESC_OCE text,
            DS_VT_ACE_OCE real
        );"""

    # Crio a tabela dimensão Tipo_Presenca
    sql_create_table_Tipo_Presenca = """
        CREATE TABLE IF NOT EXISTS Tipo_Presenca(
            tipo_presenca_ID integer PRIMARY KEY AUTOINCREMENT,
            TP_PRES integer,
            TP_PR_GER integer,
            TP_PR_OB_FG integer,
            TP_PR_DI_FG integer,
            TP_PR_OB_CE integer,
            TP_PR_DI_CE integer
        );"""

    # Crio a tabela dimensão Tipo_Sit_Quest_Disc
    sql_create_table_Tipo_Sit_Quest_Disc = """
        CREATE TABLE IF NOT EXISTS Tipo_Sit_Quest_Disc(
            tipo_sit_quest_disc_ID integer PRIMARY KEY AUTOINCREMENT,
            TP_SFG_D1 integer,
            TP_SFG_D2 integer,
            TP_SCE_D1 integer,
            TP_SCE_D2 integer,
            TP_SCE_D3 integer
        );"""

    # Crio a tabela dimensão Nota_FG_CE
    sql_create_table_Nota_FG_CE = """
        CREATE TABLE IF NOT EXISTS Nota_FG_CE(
            nota_fg_ce_ID integer PRIMARY KEY AUTOINCREMENT,
            NT_GER text,
            NT_FG text,
            NT_OBJ_FG text,
            NT_DIS_FG text,
            NT_FG_D1 integer,
            NT_FG_D1_PT integer,
            NT_FG_D1_CT integer,
            NT_FG_D2 integer,
            NT_FG_D2_PT integer,
            NT_FG_D2_CT integer,
            NT_CE text,
            NT_OBJ_CE text,
            NT_DIS_CE text,
            NT_CE_D1 integer,
            NT_CE_D2 integer,
            NT_CE_D3 integer
        );"""

    # Crio a tabela dimensão QuestioPercepProva
    sql_create_table_QuestioPercepProva = """
        CREATE TABLE IF NOT EXISTS QuestioPercepProva(
            questio_percep_prova_ID integer PRIMARY KEY AUTOINCREMENT,
            CO_RS_I1 text,
            CO_RS_I2 text,
            CO_RS_I3 text,
            CO_RS_I4 text,
            CO_RS_I5 text,
            CO_RS_I6 text,
            CO_RS_I7 text,
            CO_RS_I8 text,
            CO_RS_I9 text
        );"""

    # Crio a tabela dimensão Questio_Estudante
    sql_create_table_Questio_Estudante = """
        CREATE TABLE IF NOT EXISTS Questio_Estudante(
            questio_estudante_ID integer PRIMARY KEY AUTOINCREMENT,
            string_questionario_estudante text
        );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        create_table(conn, sql_create_table_ENADE)
        create_table(conn, sql_create_table_Tempo)
        create_table(conn, sql_create_table_Info_Curso)
        create_table(conn, sql_create_table_Estudante)
        create_table(conn, sql_create_table_NU_Item)
        create_table(conn, sql_create_table_Vetores)
        create_table(conn, sql_create_table_Tipo_Presenca)
        create_table(conn, sql_create_table_Tipo_Sit_Quest_Disc)
        create_table(conn, sql_create_table_Nota_FG_CE)
        create_table(conn, sql_create_table_QuestioPercepProva)
        create_table(conn, sql_create_table_Questio_Estudante)
    else:
        print("ERROR: cannot create the database connection.")

In [ ]:
if __name__ == '__main__':
    main()

# Questão 4) Carga de dados

## Inserindo os dados no Banco de Dados

In [ ]:
import pandas as pd

#function that insert csv data into database
def insert_data(df, table_name):
    conn = create_connection(getPath())
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Colunas de cada tabela SQL
Tempo_atributos = ['NU_ANO', 'ANO_FIM_EM', 'ANO_IN_GRAD']
Info_Curso_atributos = ['CO_IES', 'CO_CATEGAD', 'CO_ORGACAD', 'CO_GRUPO', 'CO_CURSO', 'CO_MODALIDADE', 'CO_MUNIC_CURSO', 'CO_UF_CURSO', 'CO_REGIAO_CURSO']
Estudante_atributos = ['NU_IDADE', 'TP_SEXO', 'CO_TURNO_GRADUACAO', 'TP_INSCRICAO_ADM', 'TP_INSCRICAO']
Num_Item_atributos = ['NU_ITEM_OFG', 'NU_ITEM_OFG_Z', 'NU_ITEM_OFG_X', 'NU_ITEM_OFG_N', 'NU_ITEM_OCE', 'NU_ITEM_OCE_Z', 'NU_ITEM_OCE_X', 'NU_ITEM_OCE_N']
Vetores_atributos = ['DS_VT_GAB_OFG_ORIG', 'DS_VT_GAB_OFG_FIN', 'DS_VT_GAB_OCE_ORIG', 'DS_VT_GAB_OCE_FIN', 'DS_VT_ESC_OFG', 'DS_VT_ACE_OFG', 'DS_VT_ESC_OCE', 'DS_VT_ACE_OCE']
Tipo_Presenca_atributos = ['TP_PRES', 'TP_PR_GER', 'TP_PR_OB_FG', 'TP_PR_DI_FG', 'TP_PR_OB_CE', 'TP_PR_DI_CE']
Tipo_Sit_Quest_Disc_atributos = ['TP_SFG_D1', 'TP_SFG_D2', 'TP_SCE_D1', 'TP_SCE_D2', 'TP_SCE_D3']
Nota_FG_CE_atributos = ['NT_GER', 'NT_FG', 'NT_OBJ_FG', 'NT_DIS_FG', 'NT_FG_D1', 'NT_FG_D1_PT', 'NT_FG_D1_CT', 'NT_FG_D2', 'NT_FG_D2_PT', 'NT_FG_D2_CT', 'NT_CE', 'NT_OBJ_CE', 'NT_DIS_CE', 'NT_CE_D1', 'NT_CE_D2', 'NT_CE_D3']
QuestioPercepProva_atributos = ['CO_RS_I1', 'CO_RS_I2', 'CO_RS_I3', 'CO_RS_I4', 'CO_RS_I5', 'CO_RS_I6', 'CO_RS_I7', 'CO_RS_I8', 'CO_RS_I9']
Questio_Estudante_atributos = []

In [ ]:
def insertDataENADE(dataCSV, data_pandas, cols, database_table_name):
    data_pandas = pd.read_csv(dataCSV, usecols = cols, sep = ';')
    insert_data(data_pandas, database_table_name)

def insertDataENADEAllYears(data_pandas, cols, database_table_name):
    insertDataENADE(endereco_dados_2017, data_pandas, cols, database_table_name)
    insertDataENADE(endereco_dados_2018, data_pandas, cols, database_table_name)
    insertDataENADE(endereco_dados_2019, data_pandas, cols, database_table_name)

In [ ]:
Tempo_pandas = []
Info_Curso_pandas = []
Estudante_pandas = []
Num_Item_pandas = []
Vetores_pandas = []
Tipo_Presenca_pandas = []
Tipo_Sit_Quest_Disc_pandas = []
Nota_FG_CE_pandas = []
QuestioPercepProva_pandas = []
Questio_Estudante_pandas = []

insertDataENADEAllYears(Tempo_pandas, Tempo_atributos, 'Tempo')
insertDataENADEAllYears(Info_Curso_pandas, Info_Curso_atributos, 'Info_Curso')
insertDataENADEAllYears(Estudante_pandas, Estudante_atributos, 'Estudante')
insertDataENADEAllYears(Num_Item_pandas, Num_Item_atributos, 'Num_Item')
insertDataENADEAllYears(Vetores_pandas, Vetores_atributos, 'Vetores')
insertDataENADEAllYears(Tipo_Presenca_pandas, Tipo_Presenca_atributos, 'Tipo_Presenca')
insertDataENADEAllYears(Tipo_Sit_Quest_Disc_pandas, Tipo_Sit_Quest_Disc_atributos, 'Tipo_Sit_Quest_Disc')
insertDataENADEAllYears(Nota_FG_CE_pandas, Nota_FG_CE_atributos, 'Nota_FG_CE')
insertDataENADEAllYears(QuestioPercepProva_pandas, QuestioPercepProva_atributos, 'QuestioPercepProva')
#insertDataENADEAllYears(Questio_Estudante_pandas, Questio_Estudante_atributos, 'Questio_Estudante')